# Coursework 1 - Mathematics for Machine Learning

## CID: insert your CID here

**Colab link:** insert colab link here

***
***

## Part 1: Quickfire questions [3 points]

#### Question 1 (True risk / Empirical risk):

Enter your answer here

#### Question 2 ('Large' or 'rich' hypothesis class):

Enter your answer here

#### Question 3 (Dataset splitting):

Enter your answer here

#### Question 4 (Occam’s razor):

Enter your answer here

#### Question 5 (Generalisation error):

Enter your answer here

#### Question 6 (Rademacher complexity pt1):

Enter your answer here

#### Question 7 (Rademacher complexity pt2):

Enter your answer here

#### Question 8 (Regularisation term in the loss function):

Enter your answer here

#### Question 9 (Momentum gradient descent):

Enter your answer here

#### Question 10 (Adam):

Enter your answer here

#### Question 11 (AdaGrad):

Enter your answer here

#### Question 12 (Decaying Learning Rate):

Enter your answer here

***
***

## Part 2: Short-ish proofs [6 points]


### Question 2.1: Bounds on the risk [1 point]


***

### Question 2.2: On semi-definiteness [1 point]

***

### Question 2.3: A quick recap of momentum [1 point]

***

### Question 2.4: Convergence proof [3 points]

***
***

## Part 3: A deeper dive into neural network implementations [3 points]

In [1]:
# Import libraries
import numpy as np
import torch
import torch.nn as nn
import torchvision

from torch.utils.data import DataLoader, Dataset
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import torch.optim as optim


In [2]:
# Download datasets
train_set_mnist = torchvision.datasets.MNIST(root="./data", download=True,
                                         train=True, transform=transforms.Compose([transforms.ToTensor()]));

test_set_mnist = torchvision.datasets.MNIST(root="./data",download=True,
                                        train=False,transform=transforms.Compose([transforms.ToTensor()]),);

train_set_cifar = torchvision.datasets.CIFAR10(root="./data", download=True,
                                         train=True, transform=transforms.Compose([transforms.ToTensor()]));

test_set_cifar = torchvision.datasets.CIFAR10(root="./data",download=True,
                                        train=False,transform=transforms.Compose([transforms.ToTensor()]),);

# Normalizing data:
train_set_mnist.data = nn.functional.normalize(train_set_mnist.data.to(float), p=1)
test_set_mnist.data = nn.functional.normalize(test_set_mnist.data.to(float), p=1)

Files already downloaded and verified
Files already downloaded and verified


In [3]:
class NormalizedDataSet(Dataset):
    def __init__(self, dataset_type, train: bool, normalize: bool, nclasses=10):
        match (dataset_type, train):
            case ("mnist", True):
                self.data =  train_set_mnist.data
                raw_targets = train_set_mnist.targets
                
            case ("mnist", False):
                self.data =  test_set_mnist.data
                raw_targets = test_set_mnist.targets

            case ("cifar", True):
                self.data =  train_set_cifar.data
                raw_targets = train_set_cifar.targets
                
            case ("cifar", False):
                self.data =  test_set_cifar.data
                raw_targets = test_set_cifar.targets

            case _:
                raise ValueError("Dataset must be 'mnist' or 'cifar'")
            
        if normalize and dataset_type=="mnist":                     # CIFAR is already normalized. I think.
            self.data = nn.functional.normalize(self.data, p=1)
            
        self.targets = torch.zeros(len(raw_targets), nclasses) 
        for i, t in enumerate(raw_targets):
            self.targets[i, int(t)] = 1.                            # Changing the targets into rows with 0 everywhere except the target

        self.data = torch.tensor(self.data, dtype=torch.float32)            # Casting to float to prevent future problems.
        self.targets = torch.tensor(self.targets, dtype=torch.float32)      # Just to make sure!                      
        
    def __len__(self):
        return self.data.shape[0]
    
    def __getitem__(self, index):
        return self.data[index], self.targets[index]
            
def test_dataset():
    traindata = NormalizedDataSet("cifar", False, True)
    print(traindata[10])

# test_dataset()

In [4]:
# Set seed
SEED = int('02530622')
np.random.seed(SEED)
torch.manual_seed(SEED);

In [5]:
class Net(nn.Module):
    def __init__(self, dim, nclass, width, depth):
      super().__init__()
      self.dim = dim
      self.nclass = nclass
      self.width = width
      self.depth = depth
      self.input_length = np.prod(dim)

      self.flatten = nn.Flatten()
      self.linear_in = nn.Linear(self.input_length, width)
      self.linear_hidden = nn.Linear(width, width)
      self.relu = nn.ReLU()
      self.linear_out = nn.Linear(width, nclass)

    def forward(self, x):
      flat_x = self.flatten(x)
      lifted_x = self.linear_in(flat_x)

      processed_x = lifted_x
      for _ in range(self.depth):
        processed_x = self.relu(self.linear_hidden(processed_x))

      return self.linear_out(processed_x)

def test_net(net=None):

  mnist_net = Net((28, 28), 10, 16, 2) if net is None else net
  sample_index = np.random.randint(10000)

  x = train_set_mnist.data[sample_index, :, :]
  x = torch.unsqueeze(x, 0)
  print(mnist_net(x), train_set_mnist.targets[sample_index])


# test_net()

In [14]:
class Network():
  def __init__(
      self,
      dataset_type,
      width,
      depth,
      criterion, # Notice we assume that reduction="mean"
      optimizer,
      batch_size=64,
      lr=0.001,
      max_epoch=1,
      normalize=True
      ):

    match dataset_type:
      case "mnist":
        dim = (28, 28)
        nclass = 10

      case "cifar":
        dim = (32, 32, 3)
        nclass = 10

      case _:
        raise ValueError("Dataset must be 'mnist' or 'cifar'")

    self.trainset = NormalizedDataSet(dataset_type, train=True, normalize=normalize)
    self.testset = NormalizedDataSet(dataset_type, train=False, normalize=normalize)
    self.trainloader, self.testloader = self._loading_data(batch_size, self.trainset, self.testset)

    self.net = Net(dim, nclass, width, depth)
    
    self.lr = lr
    self.max_epoch = max_epoch
    self.optimizer = optimizer(self.net.parameters(), lr=self.lr)
    self.criterion = criterion


  def _loading_data(self, batch_size, train_set, test_set):
    
    trainloader = DataLoader(train_set, batch_size, shuffle=True)
    testloader = DataLoader(test_set, batch_size, shuffle=False)

    return trainloader, testloader


  def train_epoch(self):  # Notice that all of the required arguments are now attributes!
    self.net.train()

    for X, y in self.trainloader:
      y_hat = self.net(X)
      local_loss = self.criterion(y_hat, y)

      local_loss.backward()
      self.optimizer.step()
      self.optimizer.zero_grad()
      
    

    return self.criterion(
      self.net(self.trainloader.dataset.data),
      self.trainloader.dataset.targets
      )
      
  def test_epoch(self):
    y = self.testloader.dataset.data
    targets = self.testloader.dataset.targets
    
    y_hat = self.net(y)
    mean_loss = self.criterion(y_hat, targets) # Asuuming reduction="mean"
    
    target_class = torch.max(targets, 1)[1]
    predicted_class = torch.max(y_hat, 1)[1] # Argmax gives predicted_class
    
    num_errors = len(torch.nonzero(predicted_class - target_class))
    
  
    return mean_loss, num_errors
  
  def train_me(self):
    for i in range(self.max_epoch):
      epoch = i + 1
      train_loss = self.train_epoch()
      test_loss, test_err = self.test_epoch()
      
      print(f"Epoch: {epoch} | Train Loss: {train_loss:.04} |"
            f"Test Loss: {test_loss:.04} | Test Error: {test_err}")
        
        

In [15]:

def test_train_epoch(net=None):
    mnist_net = Network(
        dataset_type="mnist",
        width=16,
        depth=2,
        criterion=nn.CrossEntropyLoss(),
        optimizer=optim.Adam,
    )
    print(mnist_net.train_epoch())

    return mnist_net

net = test_train_epoch()

def test_test_epoch():
    mnist_net = Network(
    dataset_type="mnist",
    width=16,
    depth=2,
    criterion=nn.CrossEntropyLoss(),
    optimizer=optim.Adam,
    )
    for i in range(10):
        loss, err = mnist_net.test_epoch()
        print(f"Epoch: {i+1}, Mean loss: {loss}, Errors: {err}")
        
        mnist_net.train_epoch()
        
    
# test_test_epoch()
mnist_net = Network(
    dataset_type="mnist",
    width=16,
    depth=2,
    criterion=nn.CrossEntropyLoss(),
    optimizer=optim.Adam,
    max_epoch=10
)

mnist_net.train_me()

/tmp/ipykernel_7918/526038681.py:30: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.data = torch.tensor(self.data, dtype=torch.float32)            # Casting to float to prevent future problems.
/tmp/ipykernel_7918/526038681.py:31: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.targets = torch.tensor(self.targets, dtype=torch.float32)      # Just to make sure!


tensor(0.4713, grad_fn=<DivBackward1>)
Epoch: 1 | Train Loss: 0.4758 |Test Loss: 0.4663 | Test Error: 1293
Epoch: 2 | Train Loss: 0.3892 |Test Loss: 0.3897 | Test Error: 1070
Epoch: 3 | Train Loss: 0.3468 |Test Loss: 0.3467 | Test Error: 980
Epoch: 4 | Train Loss: 0.3203 |Test Loss: 0.3281 | Test Error: 925
Epoch: 5 | Train Loss: 0.3009 |Test Loss: 0.314 | Test Error: 884
Epoch: 6 | Train Loss: 0.2779 |Test Loss: 0.2994 | Test Error: 872


In [ ]:
criterion = nn.L1Loss(reduction="none")

y_hat = torch.tensor([
    [0.2, 0., 0.5,],
    [0.4, 0.6, 0.1,],
    [0.2, 0.7, 0.5,],
    [0.9, 0.6, 0.1,]
])

y = torch.tensor([
    [1, 0., 0.,],
    [0., 1, 0.,],
    [1, 0., 0.,],
    [0., 0, 1,]
])
print(torch.max(y_hat, 1), torch.max(y, 1))
print()
print(len(torch.nonzero((torch.max(y_hat, 1)[1] - torch.max(y, 1)[1]))))

torch.return_types.max(
values=tensor([0.5000, 0.6000, 0.7000, 0.9000]),
indices=tensor([2, 1, 1, 0])) torch.return_types.max(
values=tensor([1., 1., 1., 1.]),
indices=tensor([0, 1, 0, 2]))

3


***

### Part 3.1: Implementations [1 point]

In [ ]:
# You can of course add more cells of both code and markdown. Please remember to comment the code and explain your reasoning. Include docstrings. Tutorial provide a good example of how to style your code.
# Although not compulsory you could challenge yourself by using object oriented programming to structure your code.

In [ ]:
train_dataloader, test_dataloader = loading_data(30000, train_set_mnist, test_set_mnist)
# for i, (j, k) in enumerate(train_dataloader):
#   print(f"i={i}\n\n")
#   print(f"j={j.shape}\n\n")
#   print(f"k={k}\n\n")

train_dataloader.dataset.targets


NameError: name 'loading_data' is not defined

***

### Part 3.2: Numerical exploration [2 points]

In [ ]:
# You can of course add more cells of both code and markdown.

***
***

## Part 4: The link between Neural Networks and Gaussian Processes [8 points]

### Part 4.1: Proving the relationship between a Gaussian process and a neural network [4 points]

### Task 1: Proper weight scaling

### Task 2: Derive the GP relation for a single hidden layer

### Task 3: Why in succession

### Task 4: Derive the GP relation for multiple hidden layers

***

### Part 4.2: Analysing the performance of the Gaussian process and a neural network [4 points]

In [ ]:
# Please use float64 as default dtype for this part of the assignment
torch.set_default_dtype(torch.float64)

# Another hint: when  computing [ K^L(X,X) + noise^2 Id ]^-1 y and  [ K^L(X,X) + noise^2 Id ]^-1 K^L(X,X*)
# You can TRY cholesky solve as it should be p.d. (except case for numerical errors) - maybe you can use try:/except:
# You can also try to enforce symmetry in posterior covariance by doing (K + K.t())/2

In [ ]:
# You can of course add more cells of both code and markdown.